In [ ]:
!pip install transformers sentence-transformers -q

from transformers import pipeline
from sentence_transformers import SentenceTransformer, util

def sentiment_analysis(texts):
    sentiment_analyzer = pipeline("sentiment-analysis")
    results = sentiment_analyzer(texts)
    return results

def compute_similarity(texts):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(texts, convert_to_tensor=True)
    cosine_scores = util.cos_sim(embeddings, embeddings)
    return cosine_scores

def main():
    print("📝 Enter multiple texts separated by '|||'\nFor example:\nI love this product!|||The weather is bad today|||This is great!\n")
    user_input = input("Enter texts:\n")

    # Split input by delimiter
    user_texts = [text.strip() for text in user_input.split("|||") if text.strip()]

    if len(user_texts) == 0:
        print("⚠️ No texts provided. Exiting.")
        return

    print("\n🔎 Performing Sentiment Analysis...\n")
    sentiments = sentiment_analysis(user_texts)
    for i, (text, sentiment) in enumerate(zip(user_texts, sentiments), 1):
        print(f"{i}. Text: {text}\n   Sentiment: {sentiment['label']} (score: {sentiment['score']:.4f})\n")

    if len(user_texts) > 1:
        print("🔗 Computing Text Similarities...\n")
        similarities = compute_similarity(user_texts)
        n = len(user_texts)
        for i in range(n):
            for j in range(i+1, n):
                print(f"Similarity between text {i+1} and text {j+1}: {similarities[i][j]:.4f}")
    else:
        print("⚠️ Need at least 2 texts to compute similarity.")

if __name__ == "__main__":
    main()

📝 Enter multiple texts separated by '|||'
For example:
I love this product!|||The weather is bad today|||This is great!

Enter texts:
I love this product!|||The weather is bad today|||This is great!


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.



🔎 Performing Sentiment Analysis...



Device set to use cpu


1. Text: I love this product!
   Sentiment: POSITIVE (score: 0.9999)

2. Text: The weather is bad today
   Sentiment: NEGATIVE (score: 0.9997)

3. Text: This is great!
   Sentiment: POSITIVE (score: 0.9999)

🔗 Computing Text Similarities...



...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Similarity between text 1 and text 2: -0.0125
Similarity between text 1 and text 3: 0.4800
Similarity between text 2 and text 3: 0.0221


In [ ]:
# Install required packages if not already installed
# Run this in terminal or add inside a Jupyter cell if needed
# !pip install transformers torch

from transformers import pipeline
import torch

def main():
    print("=== Text Summarization using Hugging Face Transformers ===")

    # Ask user for input text
    text = input("\nEnter the article, paper, or news content to summarize:\n")

    # Check for empty input
    if not text.strip():
        print("❗Error: No input text provided. Please try again.")
        return

    # Load the summarization pipeline with a pre-trained model
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

    try:
        # Perform summarization
        summary = summarizer(text, max_length=130, min_length=30, do_sample=False)

        # Print the summarized text
        print("\n--- Summary ---")
        print(summary[0]['summary_text'])

    except Exception as e:
        print(f"\n❗An error occurred during summarization: {e}")

if __name__ == "__main__":
    main()


=== Text Summarization using Hugging Face Transformers ===

Enter the article, paper, or news content to summarize:
In recent yeras ,US has experienced a  increse in frequency and high intensity.by this incident the United States has undergone the changes in weather conditions and the atmospheric changes.


Device set to use cpu
Your max_length is set to 130, but your input_length is only 39. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)



--- Summary ---
by this incident the United States has undergone the changes in weather conditions and the atmospheric changes. In recent yeras ,US has experienced a  increse in frequency and high intensity.


In [ ]:
!pip install langchain langchain_community transformers torch

# 1. Model Loading and Setup
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
from langchain.llms import HuggingFacePipeline # This will now be imported from langchain_community indirectly
from transformers import pipeline

# Load BlenderBot (small version for faster response)
model_name = "facebook/blenderbot-1B-distill"
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
model = BlenderbotForConditionalGeneration.from_pretrained(model_name)
blenderbot_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# 2. Wrapping the Model with LangChain
llm = HuggingFacePipeline(pipeline=blenderbot_pipeline)

# 3. Conversation Memory
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()

# 4. Prompt Template Design
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful travel assistant. Suggest places, provide mock weather, or flight info."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Build the chain
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

# 5. Chat Execution and Response Handling
import random

# Mock travel database
destinations = {
    "paris": ["Eiffel Tower", "Louvre Museum", "Seine River Cruise"],
    "tokyo": ["Mount Fuji", "Shibuya Crossing", "Tokyo Disneyland"],
    "new york": ["Statue of Liberty", "Central Park", "Broadway Show"]
}

def mock_weather(destination):
    return f"The current weather in {destination.title()} is {random.choice(['sunny', 'cloudy', 'rainy', 'snowy'])} and {random.randint(15, 30)}°C."

def mock_flight_updates():
    return "Your flight is on time and scheduled to depart at 4:30 PM."

# CLI loop
print("👋 Welcome to your LangChain Travel Assistant!")
print("Type 'exit' to quit.\n")

while True:
    user_input = input("You: ").strip().lower()
    if user_input == "exit":
        print("Assistant: Safe travels! ✈️")
        break

    # Rule-based mock responses
    responded = False
    for city in destinations:
        if city in user_input and "weather" in user_input:
            print("Assistant:", mock_weather(city))
            responded = True
            break
        elif city in user_input and "suggest" in user_input:
            print("Assistant:", f"Top attractions in {city.title()}: {', '.join(destinations[city])}.")
            responded = True
            break

    if "flight" in user_input:
        print("Assistant:", mock_flight_updates())
        responded = True

    # LLM fallback
    if not responded:
        response = chain.run(input=user_input)
        print("Assistant:", response)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


tokenizer_config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/62.9k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/2.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-8-94869c2cd6c8>:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=blenderbot_pipeline)
<ipython-input-8-94869c2cd6c8>:19: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
<ipython-input-8-94869c2cd6c8>:32: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


👋 Welcome to your LangChain Travel Assistant!
Type 'exit' to quit.

You: suggest some changes in paris and also weather update.
Assistant: The current weather in Paris is snowy and 26°C.
You: suggest some places in paris
Assistant: Top attractions in Paris: Eiffel Tower, Louvre Museum, Seine River Cruise.
You: exit
Assistant: Safe travels! ✈️


In [ ]:
from pydantic import BaseModel, ValidationError, field_validator
from datetime import datetime
from typing import Optional

class ProductData(BaseModel):
    product_id: int
    name: str
    price: float
    date_added: datetime
    rating: Optional[float]

    @field_validator("rating")
    def rating_must_be_between_0_and_5(cls, v):
        if v is not None and not (0.0 <= v <= 5.0):
            raise ValueError("Rating must be between 0.0 and 5.0")
        return v

def prompt_input():
    print("\n🛠️ Please enter product details:")

    data = {}
    try:
        data['product_id'] = int(input("👉 product_id (integer): ").strip())
        data['name'] = input("👉 name (string): ").strip()
        data['price'] = float(input("👉 price (float): ").strip())
        data['date_added'] = input("👉 date_added (YYYY-MM-DD or ISO format): ").strip()

        rating_input = input("👉 rating (optional, float 0.0 - 5.0, press Enter to skip): ").strip()
        if rating_input:
            data['rating'] = float(rating_input)
        else:
            data['rating'] = None

        # Validate with Pydantic
        validated_data = ProductData(**data)
        print("\n✅ Data validated successfully!")
        print(validated_data.model_dump_json(indent=2))  # Proper JSON with ISO date string

    except ValueError as ve:
        print(f"\n❌ Invalid input format: {ve}")
    except ValidationError as ve:
        print("\n❌ Validation errors:")
        print(ve)

if __name__ == "__main__":
    prompt_input()



🛠️ Please enter product details:
👉 product_id (integer): 202
👉 name (string): apple
👉 price (float): 238.99
👉 date_added (YYYY-MM-DD or ISO format): 2023-03-09
👉 rating (optional, float 0.0 - 5.0, press Enter to skip): 5.0

✅ Data validated successfully!
{
  "product_id": 202,
  "name": "apple",
  "price": 238.99,
  "date_added": "2023-03-09T00:00:00",
  "rating": 5.0
}
